# 00 - Data preprocessing

The following notebook is used to load the raw data, i.e., the results of wind farm control optimizations, preprocessing and finally convert it to PyTorch tensors. The raw data was not shared as was quite memory-intensive and not the main focus of this work. The preprocessed data can be downloaded at this [link](https://dtudk-my.sharepoint.com/:u:/g/personal/nicit_dtu_dk/EX96_QZYCQZMhQgLV_bEnl0BgosY2sJLT1s48-sCzHR6Cw?e=jhTNmx).

In [1]:
cd notebooks/     # go to the notebooks directory

[Errno 2] No such file or directory: 'notebooks/'
/home/nicit/deep_learning_course_wfco/notebooks


Import required packages:

In [2]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import torch

import sys
sys.path.append("../src")
import utils

Load the raw dataset of WFC optimizations and check the shapes of the keys. The data was collected in 100 recorders, each containing 960 different layouts.

In [3]:
folder_path = "/home/nicit/nordsoen/nordsoen-i-wflo/results/DL/recorders_SLSQP_simple/"

recorders = {}
n_layouts = 1000

# for each of the random states, load the recorder
for random_state in trange(int(n_layouts/10), desc="Loading recorders"):
    # load pickle
    with open(folder_path + f"recorder_{random_state}.pkl", "rb") as f:
                recorders[random_state] = pickle.load(f)

Loading recorders:   0%|          | 0/100 [00:00<?, ?it/s]

Loading recorders: 100%|██████████| 100/100 [00:21<00:00,  4.58it/s]


In [5]:
# Print all shapes
print('x_rot shape: ', recorders[5][6000]['x_rot'].shape)    # x coordinates rotated to the wind direction of 270°
print('y_rot shape: ', recorders[5][6000]['y_rot'].shape)    # y coordinates rotated to the wind direction of 270°
print('ws_eff shape: ', recorders[5][6000]['ws_eff'].shape)  # effective wind speed at each turbine
print('ti_eff shape: ', recorders[5][6000]['ti_eff'].shape)  # effective turbulence intensity at each turbine
print('ws shape: ', recorders[5][6000]['ws'].shape)          # freestream wind speed
print('yaw shape: ', recorders[5][6000]['yaw'].shape)        # yaw angles optimized with SLSQP

x_rot shape:  (31, 5)
y_rot shape:  (31, 5)
yaw shape:  (5, 31, 9)
ws_eff shape:  (5, 31, 9)
ti_eff shape:  (5, 31, 9)
ws shape:  (9,)
aep shape:  (31, 9)
aep_opt shape:  (31, 9)
yaw shape:  (5, 31, 9)


Now, for each optimization we need to extract the downwind and crosswind distance of the 5 downstream turbines, the effective wind speed, effective TI, and freestream wind speed. Then, as described in the report, we add 5 wake indexes to indicate whetherer a turbine is considered to be in the wake of the upstream turbine or not.

We also need to adjust the shape of the dataset so that we have one yaw angle (one wind turbine) for each row of the data.

In [6]:
n_layouts = 1000
rotor_diameter = 284.0              # rotor diameter of the wind turbine used for the optimizations
nearest_idx = [0, 1, 2, 3, 4]       # indices of the nearest turbines to consider (5 nearest turbines)


# --- Step 1: Load and aggregate all layouts ---
all_x_rot = []
all_y_rot = []
all_ws_eff = []
all_ti_eff = []
all_yaw = []
all_ws = []

for random_state in tqdm(range(int(n_layouts / 10)), desc="Loading recorders"):

    file_path = folder_path + f"recorder_{random_state}.pkl"
    with open(file_path, "rb") as f:
        recorder = pickle.load(f)
    for result in recorder.values():
        all_x_rot.append(result["x_rot"])       # (n_dir, n_turbines)
        all_y_rot.append(result["y_rot"])
        all_ws_eff.append(result["ws_eff"])     # (n_turbines, n_dir, n_ws)
        all_ti_eff.append(result["ti_eff"])
        all_yaw.append(result["yaw"])
        all_ws.append(result["ws"])             # (n_ws,)

# Stack everything
x_rot_all = np.stack(all_x_rot)       # (n_layouts, n_dir, n_turbines)
y_rot_all = np.stack(all_y_rot)
ws_eff_all = np.stack(all_ws_eff)     # (n_layouts, n_turbines, n_dir, n_ws)
ti_eff_all = np.stack(all_ti_eff)
yaw_all = np.stack(all_yaw)
ws_all = np.stack(all_ws)             # (n_layouts, n_ws)

n_layouts_total, n_dir, n_turbines = x_rot_all.shape
n_ws = ws_all.shape[1]
n_cases = n_layouts_total * n_dir * n_ws

# print(f"Aggregated shapes: x_rot_all={x_rot_all.shape}, ws_eff_all={ws_eff_all.shape}")

# --- Step 2: Compute dx, dy for all layouts and directions ---
# Flatten layouts and directions for _process_layout
x_rot_flat = x_rot_all.reshape(-1, n_turbines)  # (n_layouts * n_dir, n_turbines)
y_rot_flat = y_rot_all.reshape(-1, n_turbines)

dx_all, dy_all = utils._process_layout(
    turbine_x=x_rot_flat,
    turbine_y=y_rot_flat,
    rotor_diameter=rotor_diameter,
    nearest_idx=nearest_idx,
    normalize=True,
    spread=.1
)  # shape: (len(nearest_idx), n_turbines, n_layouts * n_dir)

# Repeat across wind speeds
dx_flat = np.repeat(dx_all, n_ws, axis=2).reshape(len(nearest_idx), n_turbines, n_cases)
dy_flat = np.repeat(dy_all, n_ws, axis=2).reshape(len(nearest_idx), n_turbines, n_cases)

# --- Step 3: Flatten dynamic inputs ---
ws_eff_flat = ws_eff_all.reshape(n_layouts_total, n_turbines, n_dir * n_ws)
ws_eff_flat = ws_eff_flat.transpose(1, 0, 2).reshape(n_turbines, n_cases)

ti_eff_flat = ti_eff_all.reshape(n_layouts_total, n_turbines, n_dir * n_ws)
ti_eff_flat = ti_eff_flat.transpose(1, 0, 2).reshape(n_turbines, n_cases)

yaw_flat = yaw_all.reshape(n_layouts_total, n_turbines, n_dir * n_ws)
yaw_flat = yaw_flat.transpose(1, 0, 2).reshape(n_turbines, n_cases)

# Repeat ws for all layouts and directions
# ws_rep = np.tile(ws_all.reshape(-1), n_dir)  # (n_cases,)
ws_rep = np.tile(ws_all.reshape(n_layouts_total, n_ws), (1, n_dir)).reshape(n_cases)

# --- Step 4: Build feature matrix in one shot ---
dx_part = dx_flat.transpose(1, 2, 0)  # (n_turbines, n_cases, 5)
dy_part = dy_flat.transpose(1, 2, 0)

# modify this so that it is -1 when it is not waked (or waked), instaed of zero!
waked_part = np.where(dx_part != np.inf, 1.0, -1.0)

# Replace inf with nan for dx/dy
dx_part[dx_part == np.inf] = np.nan

# when there is nan in the corresponding dx, set dy to nan as well
dy_part[dy_part == np.inf] = np.nan

# Ensure dy is NaN wherever dx is NaN
dy_part[np.isnan(dx_part)] = np.nan

# adjust so that whenever the ws_eff is >= 12, the yaw should be zero
yaw_flat = np.where((ws_eff_flat >= 10.5) 
                    # & (np.abs(yaw_flat) == 25.0)
                    , 0.0, yaw_flat)

# same whenever ws_eff is < 4
yaw_flat = np.where(ws_eff_flat < 4.0, 0.0, yaw_flat)

# round the yaw to the nearest float with 1 decimal place
yaw_flat = np.round(yaw_flat, 1)

# Static features: dx, dy, waked flags
static_features = np.concatenate([dx_part, dy_part, 
                                waked_part
                                  ], axis=2)  # (n_turbines, n_cases, 15)

# merge ws and ws_eff into one feature (the difference)
dynamic_features = np.stack([
    ws_eff_flat,
    ti_eff_flat,
    np.tile(ws_rep, (n_turbines, 1))
], axis=2)

# Combine all features
X = np.concatenate([static_features, dynamic_features], axis=2)  # (n_turbines, n_cases, 18)
X = X.reshape(-1, X.shape[2])  # (n_turbines * n_cases, 18)
Y = yaw_flat.reshape(-1)

# Convert to torch tensors
X_torch = torch.tensor(X, dtype=torch.float32)
Y_torch = torch.tensor(Y, dtype=torch.float32)

print(f"Final dataset shapes: X = {X_torch.shape}, Y = {Y_torch.shape}")

# Save dataset
torch.save((X_torch, Y_torch), f"../data/5wt_dataset_{n_layouts}_slsqp_simple_complete.pt")
print(f"Dataset saved to 5wt_dataset_{n_layouts}_slsqp_simple_complete.pt")


Loading recorders: 100%|██████████| 100/100 [00:07<00:00, 13.60it/s]


Final dataset shapes: X = torch.Size([133920000, 18]), Y = torch.Size([133920000])
Dataset saved to 5wt_dataset_1000_slsqp_simple_complete.pt
